In [3]:
import wfdb
import pandas as pd

def convert_physionet_data(dat_file, hea_file, xyz_file=None):
    # 读取头文件以获取元数据信息
    record = wfdb.rdrecord(dat_file, pn_dir='')
    
    # 读取.xyz文件（如果存在）
    if xyz_file:
        with open(xyz_file, 'r') as f:
            xyz_data = f.read()
    else:
        xyz_data = None
    
    # 读取数据
    signals, fields = wfdb.rdsamp(dat_file, pn_dir='')
    
    # 转换为DataFrame
    df = pd.DataFrame(signals, columns=fields['sig_name'])
    
    # 添加时间列
    df['Time'] = pd.date_range(start='00:00', periods=len(df), freq=str(1/fields['fs'])+'S')
    
    return df, record.__dict__, xyz_data

# 示例用法
dat_file = 'ecg/patient001/s0010_re.dat'
hea_file = 'ecg/patient001/s0010_re.hea'
xyz_file = 'ecg/patient001/s0010_re.xyz'  # 如果没有.xyz文件，可以将其设置为None

df, metadata, xyz_data = convert_physionet_data(dat_file, hea_file, xyz_file)

# 将数据保存为CSV文件
df.to_csv('converted_data.csv', index=False)

# 显示元数据信息和.xyz数据
print("Metadata:", metadata)
print("XYZ Data:", xyz_data)

NetFileNotFoundError: 404 Error: Not Found for url: https://physionet.org/files/1.0.0/s0010_re.dat.hea

In [5]:
!pwd

/root


In [10]:
import wfdb
import numpy as np
import pandas as pd

def convert_physionet_to_csv(dat_file, hea_file, csv_file):
    # 读取数据
    signals, fields = wfdb.rdsamp(dat_file)
    
    # 读取头文件
    record = wfdb.rdheader(hea_file)
    num_samples = record.sig_len
    sampling_frequency = record.fs 

    print(sampling_frequency)
    
    # 构建DataFrame
    df = pd.DataFrame(signals, columns=record.sig_name)
    df.index = np.arange(1, num_samples + 1)
    
    # 写入CSV文件
    df.to_csv(csv_file, index_label='Time (ms)')

# 替换为你下载的数据文件的路径
dat_file = '/root/ecg/patient001/s0010_re'
hea_file = '/root/ecg/patient001/s0010_re'
xyz_file = '/root/ecg/patient001/s0010_re'

csv_file = 'output_csv_file.csv'

convert_physionet_to_csv(dat_file, hea_file, csv_file)


1000


In [17]:
import requests
import csv

# 定义请求的URL
url = 'http://10.1.147.25:11434/api/generate'
csvfile = 'output_csv_file.csv'

# 读取 CSV 文件内容
with open(csvfile, 'r') as file:
    reader = csv.DictReader(file)
    # 将 CSV 数据转换为列表字典
    data_list = list(reader)

prompt = ("请帮我分析这份心电图的数据，它包含了时间（以毫秒为单位）和多个心电图导联"
          "（i, ii, iii, avr, avl, avf, v1, v2, v3, v4, v5, v6, vx, vy, vz）。"
          "我想了解每个导联的波形特征，包括但不限于波形形状、振幅、频率等。")

# 定义要发送的JSON数据
data = {
    "model": "codellama:13b",
    "prompt": prompt,
    "format": "json",
    "stream": False,
    "data": data_list  # 将 CSV 数据作为请求的数据
}
# 发送POST请求
response = requests.post(url, json=data)

# 检查响应状态码
if response.status_code == 200:
    # 请求成功，输出响应内容
    print(response.json())
else:
    # 请求失败，输出错误信息
    print("Error:", response.status_code)


{'model': 'codellama:13b', 'created_at': '2024-06-04T08:16:34.588362711Z', 'response': '{\n"data": {\n"time": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],\n"lead_i": [1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 1.10],\n"lead_ii": [2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 2.7, 2.8, 2.9, 2.10],\n"lead_iii": [3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9, 3.10],\n"lead_avr": [4.1, 4.2, 4.3, 4.4, 4.5, 4.6, 4.7, 4.8, 4.9, 4.10],\n"lead_avl": [5.1, 5.2, 5.3, 5.4, 5.5, 5.6, 5.7, 5.8, 5.9, 5.10],\n"lead_avf": [6.1, 6.2, 6.3, 6.4, 6.5, 6.6, 6.7, 6.8, 6.9, 6.10],\n"lead_v1": [7.1, 7.2, 7.3, 7.4, 7.5, 7.6, 7.7, 7.8, 7.9, 7.10],\n"lead_v2": [8.1, 8.2, 8.3, 8.4, 8.5, 8.6, 8.7, 8.8, 8.9, 8.10],\n"lead_v3": [9.1, 9.2, 9.3, 9.4, 9.5, 9.6, 9.7, 9.8, 9.9, 9.10],\n"lead_v4": [10.1, 10.2, 10.3, 10.4, 10.5, 10.6, 10.7, 10.8, 10.9, 10.10],\n"lead_v5": [11.1, 11.2, 11.3, 11.4, 11.5, 11.6, 11.7, 11.8, 11.9, 11.10],\n"lead_v6": [12.1, 12.2, 12.3, 12.4, 12.5, 12.6, 12.7, 12.8, 12.9, 12.10],\n"lead_vx": [13.1, 13.2, 13.3, 13.4, 13.5, 1